## Setting up environment

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

In [ ]:
import h5py
import numpy as np

In [ ]:
from ctapipe.instrument import CameraGeometry
from ctapipe.image.geometry_converter import convert_geometry_hex1d_to_rect2d
from ctapipe.calib import CameraCalibrator
from ctapipe.io import event_source
from ctapipe.utils import get_dataset_path

In [ ]:
filename = get_dataset_path('gamma_test_large.simtel.gz')
source = event_source(filename)
calib = CameraCalibrator()

In [ ]:
maxevents = 2       # max number of events to save in hdf5 file
imagestels = {}      # tels  dict where 2d images will be stored
imagesevts = {}      # event dict where 2d images will be stored
count = 1

## Creation of 2D images

In [ ]:
for event in tqdm(source):

    count +=1

    # calibrating images
    calib.calibrate(event)

    for tel_id, dl1cameracontainer in event.dl1.tel.items():
        telname = "tel_{:03d}".format(tel_id)

        # create images with camera geometry
        camgeom = event.inst.subarray.tel[tel_id].camera

        signal = dl1cameracontainer.image[0] # let's take channel 0
        new_geom, image2d = convert_geometry_hex1d_to_rect2d(camgeom, signal)

        # store 2d images in dicts 
        #
        datasetname = '/DL1/'+telname
        eventname = str(event.r0.event_id)
        
        if datasetname not in imagestels:
            imagestels[datasetname] = []
            
        if eventname not in imagesevts:
            imagesevts[eventname] = []    

        imagestels[datasetname].append(image2d)
        imagesevts[eventname].append(image2d)       

    if count>maxevents: break

## Creation of HDF5 file

In [ ]:
h5f = h5py.File('tableimages.h5', 'w')
for key in imagestels:
    h5f.create_dataset(key, data=imagestels[key])
h5f.close()

## Generated HDF5 file

* [DL1 Images Table Dataset](tableimages.h5)

## 3D inspection for a given telescope

In [ ]:
#for key in imagestels:
#    print(key, len(imagestels[key]))

In [ ]:
# datacube = load_hdf5_data_from_somewhere()  # assuming shape is (N, X, Y)
# no need to load fiie 
datacube = np.array(imagestels['/DL1/tel_028'])

@interact
def displaycube(event=(0, len(datacube)-1)):
    plt.imshow(datacube[event])
    plt.show()

## 3D inspection for a given event

In [ ]:
#for key in imagesevts:
#    print(key, len(imagesevts[key]))

In [ ]:
# datacube = load_hdf5_data_from_somewhere()  # assuming shape is (N, X, Y)
# no need to load fiie 
datacube = np.array(imagesevts['23703'])

@interact
def displaycube(tel=(0, len(datacube)-1)):
    plt.imshow(datacube[tel])
    plt.show()